In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os
import random
import shutil

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, adjusted_rand_score
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical, set_random_seed
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from pyspark.sql.functions import col, udf, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

2024-09-20 13:05:46.470586: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 13:05:46.530406: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 13:05:46.612434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 13:05:46.681716: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 13:05:46.702772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 13:05:46.835987: I tensorflow/core/platform/cpu_feature_gu

## En local sur mon ordinateur, sans PySpark

In [2]:
base_model = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3))

In [3]:
def create_model(base_model, nbr_de_classes):
    for layer in base_model.layers:
        layer.trainable = False
    # Ajout des couches de classification
    model = Sequential([base_model,
                        GlobalAveragePooling2D(),
                        Dense(1024, activation='relu'),
                        Dense(nbr_de_classes, activation='softmax')
                       ])
    return model

In [4]:
model = create_model(base_model, 3)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,572,803 (13.63 MB)

 Trainable params: 1,314,819 (5.02 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:
def metrics_display(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')
    recall = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')
    ARI = adjusted_rand_score(true_labels, predicted_labels)
    print(f'accuracy = {accuracy}   precision = {precision}   recall={recall}   f1 = {f1}   ARI = {ARI}')

In [39]:
def train_and_test_model(train_dir,
                         test_dir,
                         model,
                         seed_value=42,
                         target_size=(224, 224),
                         optimizer=Nadam,
                         learning_rate=0.001,
                         epochs=10
                        ):
    
    # Fixation des germes des générateurs aléatoires pour assurer la reproductibilité des résultats.
    set_random_seed(seed_value)
    
    # Préparation du générateur de données
    generator = ImageDataGenerator(rescale=1./255)
    
    train_generator = generator.flow_from_directory(train_dir,
                                                    target_size=target_size,
                                                    class_mode='categorical',
                                                    seed=seed_value
                                                    )

    test_generator = generator.flow_from_directory(test_dir,
                                                   target_size=target_size,
                                                   class_mode='categorical',
                                                   shuffle=False,
                                                   seed=seed_value
                                                   )
    
    # Compiler le modèle
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Construction des callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

    # Entraînement du modèle
    history = model.fit(train_generator,
                        validation_data=test_generator,
                        callbacks=[early_stopping, reduce_lr],
                        epochs=epochs)

    # Obtention des étiquettes prédites et réelles
    predictions = model.predict(test_generator)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes
    
    # Évaluation des performances du modèle
    metrics_display(y_true, y_pred)

In [4]:
train_dir = '../data/train_local'
test_dir = '../data/test_local'

In [44]:
train_and_test_model(train_dir, test_dir, model)

Found 1381 images belonging to 3 classes.
Found 461 images belonging to 3 classes.
Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 44s 920ms/step - accuracy: 0.8996 - loss: 0.1968 - val_accuracy: 1.0000 - val_loss: 1.2202e-04 - learning_rate: 0.0010
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 906ms/step - accuracy: 1.0000 - loss: 2.9620e-05 - val_accuracy: 1.0000 - val_loss: 6.4307e-05 - learning_rate: 0.0010
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 902ms/step - accuracy: 1.0000 - loss: 6.8215e-06 - val_accuracy: 1.0000 - val_loss: 3.1388e-05 - learning_rate: 0.0010
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 44s 996ms/step - accuracy: 1.0000 - loss: 3.3063e-06 - val_accuracy: 1.0000 - val_loss: 2.3720e-05 - learning_rate: 0.0010
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 41s 925ms/step - accuracy: 1.0000 - loss: 2.1207e-06 - val_accuracy: 1.0000 - val_loss: 1.6217e-05 - learning_rate: 5.0000e-04
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 900ms/step - accuracy: 1.0000 - loss: 1.4989e-06 - val_accuracy: 1.000

## En local sur mon ordinateur, avec PySpark

In [5]:
env_projet = '/home/chakir/.local/share/virtualenvs/Projet8-N0cGnpCf/bin/python'

In [6]:
# Configuration de l'environnement virtuel pour PySpark
os.environ['PYSPARK_PYTHON'] = env_projet
os.environ['PYSPARK_DRIVER_PYTHON'] = env_projet

In [7]:
# Démarrage d'une session Spark
spark = (SparkSession.builder
                     .appName('P8')
                     .master('local')
                     .config('spark.executorEnv.PYSPARK_PYTHON', env_projet)
                     .getOrCreate())

24/09/17 14:03:21 WARN Utils: Your hostname, chakir-Latitude-7390 resolves to a loopback address: 127.0.1.1; using 192.168.43.175 instead (on interface wlp2s0)
24/09/17 14:03:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/17 14:03:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/17 14:03:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/17 14:03:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
sc = spark.sparkContext

In [9]:
spark

In [11]:
images_train = (spark.read.format('binaryFile')
                     .option('pathGlobFilter', '*.jpg')
                     .option('recursiveFileLookup', 'true')
                     .load(train_dir))

images_test = (spark.read.format('binaryFile')
                    .option('pathGlobFilter', '*.jpg')
                    .option('recursiveFileLookup', 'true')
                    .load(test_dir))

In [12]:
images_train = images_train.withColumn('label', element_at(split(images_train['path'], '/'),-2))
images_test = images_test.withColumn('label', element_at(split(images_test['path'], '/'),-2))

In [13]:
print(images_train.printSchema())
print(images_test.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------------------------------------+--------------+
|path                                                                                 |label         |
+-------------------------------------------------------------------------------------+--------------+
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_326_100.jpg|Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_4_100.jpg  |Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_8_100.jpg  |Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_324_100.jpg|Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeb

In [10]:
def create_model_spark(spark_context, base_model, nbr_de_classes):
    for layer in base_model.layers:
        layer.trainable = False
    # Ajout des couches de classification
    model = Sequential([base_model,
                        GlobalAveragePooling2D(),
                        Dense(1024, activation='relu'),
                        Dense(nbr_de_classes, activation='softmax')
                       ])
    brodcast_weights = spark_context.broadcast(model.get_weights())
    return model

In [11]:
# Fonction de prétraitement d'images (redimensionnement, normalisation)
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array /= 255.0
    return img_array

# Fonction pour obtenir les chemins des images et leurs étiquettes
def get_image_paths_and_labels(spark_session, dir_path):
    
    images = (spark_session.read.format('binaryFile')
                           .option('pathGlobFilter', '*.jpg')
                           .option('recursiveFileLookup', 'true')
                           .load(dir_path))

    images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
    
    return images

In [37]:
def train_and_test_model_spark(train_dir, 
                               test_dir, 
                               model,
                               spark_session,
                               seed_value=42, 
                               target_size=(224, 224), 
                               optimizer=Nadam, 
                               learning_rate=0.001, 
                               epochs=10):
    
    # Fixation des germes pour la reproductibilité des résultats
    set_random_seed(seed_value)
    
    # Obtention des chemins des images et leurs étiquettes
    train_df = get_image_paths_and_labels(spark_session, train_dir)
    test_df = get_image_paths_and_labels(spark_session, test_dir)
    
    # Conversion en dataframe pandas pour prétraitement en local
    train_data = train_df.toPandas()
    test_data = test_df.toPandas()

    # Nettoyage des chemins de fichiers en retirant le préfixe "file:"
    train_data['path'] = train_data.path.apply(lambda x: x.replace('file:', ''))
    test_data['path'] = test_data.path.apply(lambda x: x.replace('file:', ''))
    
    # Prétraitement des images
    X_train = np.array([preprocess_image(path) for path in train_data.path])
    X_test = np.array([preprocess_image(path) for path in test_data.path])

    # Sélection des étiquettes d'entraînement et de test
    y_train = train_data.label
    y_test = test_data.label
    
    # Encodage des étiquettes en one-hot
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train.values)
    y_test = encoder.transform(y_test.values)
    y_train_cnn = to_categorical(y_train)
    y_test_cnn = to_categorical(y_test)

    # Compiler le modèle
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Callbacks pour le modèle
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
    
    # Entraîner le modèle
    history = model.fit(X_train, y_train_cnn,
                        validation_data=(X_test, y_test_cnn),
                        callbacks=[early_stopping, reduce_lr],
                        epochs=epochs)
    
    # Prédictions
    predictions = model.predict(X_test)
    y_pred = np.argmax(predictions, axis=1)
    
    # Évaluation du modèle
    metrics_display(y_test, y_pred)

In [38]:
model = create_model_spark(sc, base_model, 3)

In [39]:
train_and_test_model_spark(train_dir, test_dir, model, spark)

Epoch 1/10


2024-09-17 14:41:02.977600: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 831516672 exceeds 10% of free system memory.


44/44 ━━━━━━━━━━━━━━━━━━━━ 47s 986ms/step - accuracy: 0.8898 - loss: 0.2945 - val_accuracy: 1.0000 - val_loss: 3.2307e-04 - learning_rate: 0.0010
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 41s 928ms/step - accuracy: 1.0000 - loss: 5.2905e-05 - val_accuracy: 1.0000 - val_loss: 1.7070e-04 - learning_rate: 0.0010
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 42s 957ms/step - accuracy: 1.0000 - loss: 1.9368e-05 - val_accuracy: 1.0000 - val_loss: 1.0505e-04 - learning_rate: 0.0010
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 43s 976ms/step - accuracy: 1.0000 - loss: 1.0073e-05 - val_accuracy: 1.0000 - val_loss: 7.2057e-05 - learning_rate: 0.0010
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 1.0000 - loss: 6.2570e-06 - val_accuracy: 1.0000 - val_loss: 4.5939e-05 - learning_rate: 0.0010
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 1.0000 - loss: 4.2515e-06 - val_accuracy: 1.0000 - val_loss: 3.8880e-05 - learning_rate: 0.0010
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 41s 936ms/step - 

## Sur le cloud AWS

In [3]:
import numpy as np

# Données
T0 = 70  # Température initiale (°C)
Tamb = 10  # Température ambiante (°C)
thickness = 0.025  # Épaisseur de la plaque (m)
k_thermal = 0.7  # Conductivité thermique (W/m·K)
c_specific = 800  # Capacité thermique massique (J/kg·K)
density = 2000  # Densité de la terre cuite (kg/m^3)

# Constante de convection approximative (basée sur une convection naturelle d'air à l'intérieur)
h_conv = 10  # W/m²K (hypothèse)

# Surface de la plaque (en supposant une plaque de 2m x 1.5m)
length = 2  # m
width = 1.5  # m
area = length * width  # m²

# Volume et masse de la plaque
volume = area * thickness  # m³
mass = density * volume  # kg

# Constante de refroidissement approximative
k = (h_conv * area) / (mass * c_specific)  # Constante de refroidissement

# Temps estimé pour atteindre Tamb + epsilon proche (par exemple à 16°C)
def temperature(t, T0, Tamb, k):
    return Tamb + (T0 - Tamb) * np.exp(-k * t)

# Cherchons le temps pour que la température atteigne 16°C (~refroidissement quasi complet)
target_temp = 16
time = np.linspace(0, 10 * 3600, 10000)  # temps en secondes

# Calcul de la température au cours du temps
temperatures = temperature(time, T0, Tamb, k)

# Trouvons le temps où la température atteint 16°C
time_to_cool = time[np.argmin(np.abs(temperatures - target_temp))]  # en secondes

# Convertissons en heures
time_to_cool_hours = time_to_cool / 3600
time_to_cool_hours

2.5582558255825583